<a href="https://colab.research.google.com/github/imaabay/CA2_Repository/blob/main/method1/gpt_experiments/ST01A_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basic Template

In [ ]:
!pip install langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain import PromptTemplate
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")

In [ ]:
from langsmith import Client

client = Client()

## Setup Evaluation Metrics

In [ ]:
from langchain_openai import ChatOpenAI # Try ChatAnthropic as well
from langchain_core.prompts.prompt import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator

_PROMPT_TEMPLATE = """
  You are an expert tasked with evaluating the explainability of large language model-generated answers for medical diagnoses.
  Your role is to assess whether the given answers provide sufficient explanation and clarity for a user to understand the medical diagnosis.
  You are assessing the following question:
  {query}
  Here is the real answer:
  {context}
  You are assessing the following predicted answer:
  {result}
"""

PROMPT = PromptTemplate(
    input_variables = ["query", "context", "result"],
    template = _PROMPT_TEMPLATE
)

eval_llm = ChatOpenAI( temperature=0)

evaluators = [
  LangChainStringEvaluator("context_qa", config={"llm": eval_llm, "prompt": PROMPT}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "conciseness"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "coherence"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "detail"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "helpfulness"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "depth"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "insensitivity"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "harmfulness"}),

]

## Run Evaluation

In [ ]:
import openai

openai_client = openai.Client()

def my_app_v1(question):
   print("question: ", question)
   return openai_client.chat.completions.create(
       model="gpt-4o-mini",
       temperature=0,
       messages=[
           {
               "role": "system",
               "content": "You are an AI chatbot designed to assist doctors in diagnosing patients. Your role is to provide accurate diagnosis based on the information provided and to clearly explain the reasoning behind each diagnosis. Always ensure that the information you provide is truthful and reliable."
           },
           {
               "role": "user",
               "content": question,
           }
       ],
   ).choices[0].message.content

def langsmith_app(inputs):
  print("inputs: ", inputs)
  output = my_app_v1(inputs["question"])
  print("output: ", output)
  return {"output": output}

In [ ]:
from langsmith import evaluate

experiment_results = evaluate(
    langsmith_app, # AI System
    data = "XAI_EVAL",
    evaluators=evaluators,
    experiment_prefix="REMOVE-TESTONLY-basic-template-evaluation-gpt-4o-mini"
)

View the evaluation results for experiment: 'REMOVE-TESTONLY-basic-template-evaluation-gpt-4o-mini-79149ab7' at:
https://smith.langchain.com/o/683c5cb9-3b64-5127-a5a8-405b032642f2/datasets/b7b8dceb-8703-42f2-92ba-cfa8aad7542b/compare?selectedSessions=9513a8fd-882d-4724-849b-69a415f8be44




0it [00:00, ?it/s]

inputs:  {'question': 'What are some common causes of one sided SI joint Pain that comes and goes? I am 25 and just had a baby (naturally) 10 months ago. Back pain started at 8 months Postpartum .'}
question:  What are some common causes of one sided SI joint Pain that comes and goes? I am 25 and just had a baby (naturally) 10 months ago. Back pain started at 8 months Postpartum .
inputs: inputs:  {'question': 'Hi,I am 16 years old female. I urge to pee quite often. I have a sharp pain in my vagina when I pee. I have the urge to pee frequently. But when I go to the toilet, I do not have the urge to pee. Is this due to UTI or STD?'}
 {'question': 'Hello, I am just now noticing these patches on my skin and would like your input. My skin does not itch but I have been noticing patches on my chest and it is all over my back.'}
question:  Hello, I am just now noticing these patches on my skin and would like your input. My skin does not itch but I have been noticing patches on my chest and it


https://huggingface.co/PrunaAI/OpenBioLLM-Llama3-70B-GGUF-smashed


 -- CUDA host compiler is GNU
  CMake Error at vendor/llama.cpp/ggml/src/CMakeLists.txt:1010 (get_flags):
    get_flags Function invoked with incorrect arguments for function named:
    get_flags